In [368]:
from model.bert import bert_ATE
from data.dataset import dataset_ATM
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import pickle
import os

pretrain_model_name = 'bert-base-uncased'
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    pols_tensors = [s[3] for s in samples]
    pols_tensors = pad_sequence(pols_tensors, batch_first=True)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
    
    return ids_tensors, tags_tensors, pols_tensors, masks_tensors


test_ds=dataset_ATM(pd.read_csv("data/device/split2/test2.csv"), tokenizer)
test_loader = DataLoader(test_ds, batch_size=16, collate_fn=create_mini_batch, shuffle = True)

def tags_to_spans(tags_list):
    spans_list = []
    for tags in tags_list:
        spans = []
        start = None
        for i, tag in enumerate(tags):
            if tag == 1:  # 属性开始
                if start is not None:
                    spans.append((start, i - 1))
                start = i
            elif tag == 0:  # 不属于属性
                if start is not None:
                    spans.append((start, i - 1))
                    start = None
        if start is not None:
            spans.append((start, len(tags) - 1))
        spans_list.append(spans)
    return spans_list

def test_model_ATE(loader):
    pred = []
    trueth = []
    with torch.no_grad():
        for data in loader:

            ids_tensors, tags_tensors, _, masks_tensors = data
            ids_tensors = ids_tensors.to(DEVICE)
            tags_tensors = tags_tensors.to(DEVICE)
            masks_tensors = masks_tensors.to(DEVICE)

            outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

            for i in range(len(outputs)):
                # 将每个句子的预测结果转换为整数，并添加到 pred 列表中
                pred.append([int(j) for j in outputs[i]])
                # 将每个句子的真实标签转换为整数，并添加到 trueth 列表中
                trueth.append([int(j) for j in tags_tensors[i]])

    pred_spans = tags_to_spans(pred)
    trueth_spans = tags_to_spans(trueth)

    return trueth_spans, pred_spans

model_ATE = load_model(model_ATE, 'pkl/rd333-2.pkl')

def match(pred, gold):
    true_count = 0
    for t in pred:
        if t in gold:
            true_count += 1
    return true_count

def evaluate(pred_spans, trueth_spans):
    total_true_count = 0
    total_pred_count = 0
    total_gold_count = 0
    
    for pred, gold in zip(pred_spans, trueth_spans):
        true_count = match(pred, gold)
        total_true_count += true_count
        total_pred_count += len(pred)
        total_gold_count += len(gold)
        
    precision = total_true_count / total_pred_count if total_pred_count > 0 else 0
    recall = total_true_count / total_gold_count if total_gold_count > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    
    return precision, recall, f1

# 使用 test_model_ATE 函数获取预测的属性列表和真实的属性列表
trueth_spans, pred_spans = test_model_ATE(test_loader)

# 使用 evaluate 函数计算匹配的准确率、召回率和 F1 分数
precision, recall, f1 = evaluate(pred_spans, trueth_spans)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1: {f1:.4f}')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Precision: 0.4914
Recall: 0.3957
F1: 0.4384


In [1]:
from model.bert import bert_ATE
from data.dataset import dataset_ATM
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import pickle
import os

pretrain_model_name = 'bert-base-uncased'
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(model_ATE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert_ATE(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   